In [ ]:
using NeuralPDE, Lux, ModelingToolkit, Optimization, OptimizationOptimJL
using ModelingToolkit: Interval
using Plots



## Решение волнового уравнения на плоскости

In [ ]:
@parameters x y
@variables u(..)
Dxx = Differential(x)^2
Dyy = Differential(y)^2

# 2D PDE
eq = Dxx(u(x, y)) + Dyy(u(x, y)) ~ -sin(pi * x) * sin(pi * y)

# Boundary conditions
bcs = [u(0, y) ~ 0.0, u(1, y) ~ 0.0,
    u(x, 0) ~ 0.0, u(x, 1) ~ 0.0]
# Space and time domains
domains = [x ∈ Interval(0.0, 1.0),
    y ∈ Interval(0.0, 1.0)]

In [ ]:
# Neural network
dim = 2 # number of dimensions
chain = Chain(Dense(dim, 16, σ), Dense(16, 16, σ), Dense(16, 1))

In [ ]:
discretization = PhysicsInformedNN(
    chain, QuadratureTraining(; batch = 200, abstol = 1e-6, reltol = 1e-6))

In [ ]:
@named pde_system = PDESystem(eq, bcs, domains, [x, y], [u(x, y)])
prob = discretize(pde_system, discretization)

In [ ]:
#Optimizer
opt = OptimizationOptimJL.LBFGS(linesearch = BackTracking())

callback = function (p, l)
    println("Current loss is: $l")
    return false
end

# We can pass the callback function in the solve. Not doing here as the output would be very long.
res = Optimization.solve(prob, opt, maxiters = 1000, callback = callback)
phi = discretization.phi

In [ ]:
dx = 0.05
xs, ys = [infimum(d.domain):(dx / 10):supremum(d.domain) for d in domains]
analytic_sol_func(x, y) = (sin(pi * x) * sin(pi * y)) / (2pi^2)

u_predict = reshape([first(phi([x, y], res.u)) for x in xs for y in ys],
    (length(xs), length(ys)))
u_real = reshape([analytic_sol_func(x, y) for x in xs for y in ys],
    (length(xs), length(ys)))
diff_u = abs.(u_predict .- u_real)

p1 = plot(xs, ys, u_real, linetype = :contourf, title = "analytic");
p2 = plot(xs, ys, u_predict, linetype = :contourf, title = "predict");
p3 = plot(xs, ys, diff_u, linetype = :contourf, title = "error");
plot(p1, p2, p3)

## Решение уравнения для заряда на плоскости

In [ ]:
using NeuralPDE, Lux, ModelingToolkit, Optimization, OptimizationOptimJL, LineSearches,
      OptimizationOptimisers, LuxCUDA, Random, ComponentArrays
using ModelingToolkit: Interval, infimum, supremum
using Distributions

In [ ]:
@parameters t, x, y, z
@variables E(..)[1:3], B(..)[1:3]

In [ ]:
Dt = Differential(t)
Dtt = Differential(t)^2
Dx = Differential(x)
Dxx = Differential(x)^2
Dy = Differential(y)
Dxy = Differential(x) * Differential(y)
Dyy = Differential(y)^2
Dz = Differential(z)
Dxz = Differential(x) * Differential(z)
Dzz = Differential(z)^2
Dyz = Differential(y) * Differential(z)

In [ ]:
using LinearAlgebra, Symbolics

In [ ]:
function divergence(f)
    #return sum(Symbolics.sparsejacobian_vals(f, [x; y; z], [1; 2; 3], [1; 2; 3]))
    return Dx(f[1]) + Dy(f[2]) + Dz(f[3])
end
function rotor(f)
    #J = Symbolics.jacobian(f, [x; y; z])
    #ex = J[3, 2] - J[2, 3] Dz(f[2]) - Dy(f[3])
    #ey = J[1, 3] - J[3, 1] Dz(f[2]) - Dy(f[3])
    #ez = J[2, 1] - J[1, 2]
    ex = Dz(f[2]) - Dy(f[3])
    ey = Dx(f[3]) - Dz(f[1])
    ez = Dy(f[1]) - Dx(f[2])
    return [ex; ey; ez]
end

In [ ]:
@parameters t, x, y, z
@variables E(..)[1:3], B(..)[1:3]

charge = Distributions.MvNormal([0.5; 0.5; 0.5], [1e-3 0.0 0.0; 0.0 1e-3 0.0; 0.0 0.00 1e-3])
rho = (x, y, z) -> Distributions.pdf(charge, [x; y; z])


In [ ]:
# 3D PDE
eqs = [
    divergence(E(x, y, z)) ~ 4 * pi * rho(x, y, z),
    divergence(B(x, y, z)) ~ 0.0,
    rotor(E(x, y, z)) ~ [0.0; 0.0; 0.0],
    rotor(B(x, y, z)) ~ [0.0; 0.0; 0.0],
]

In [ ]:




# Boundary conditions
bcs = [E(0, y, z) ~ [0.0; 0.0; 0.0], E(1, y, z) ~ [0.0; 0.0; 0.0], 
    E(x, 0, z) ~ [0.0; 0.0; 0.0], E(x, 1, z) ~ [0.0; 0.0; 0.0],
    E(x, y, 0) ~ [0.0; 0.0; 0.0], E(x, y, 0) ~ [0.0; 0.0; 0.0],
    B(0, y, z) ~ [0.0; 0.0; 0.0], B(1, y, z) ~ [0.0; 0.0; 0.0], 
    B(x, 0, z) ~ [0.0; 0.0; 0.0], B(x, 1, z) ~ [0.0; 0.0; 0.0],
    B(x, y, 0) ~ [0.0; 0.0; 0.0], B(x, y, 0) ~ [0.0; 0.0; 0.0]]
# Space and time domains
domains = [x ∈ Interval(0.0, 1.0),
    y ∈ Interval(0.0, 1.0),
    z ∈ Interval(0.0, 1.0)]

In [ ]:
# Neural network
input_ = 3
n = 30
dim = 3 # number of dimensions
chain = [Chain(Dense(input_, n, σ), Dense(n, n, σ), Dense(n, dim)) for _ in 1:2]

In [ ]:
strategy = StochasticTraining(128)
discretization = PhysicsInformedNN(chain, StochasticTraining(128))

In [ ]:
@variables u(..)

In [ ]:
@named pde_system = PDESystem(eqs, bcs, domains, [x, y, z], 
    [E(x, y, z), B(x, y, z)])

In [ ]:
prob = discretize(pde_system, discretization)
sym_prob = symbolic_discretize(pdesystem, discretization)

In [ ]:
pde_inner_loss_functions = sym_prob.loss_functions.pde_loss_functions
bcs_inner_loss_functions = sym_prob.loss_functions.bc_loss_functions

In [ ]:
callback = function (p, l)
    println("loss: ", l)
    println("pde_losses: ", map(l_ -> l_(p.u), pde_inner_loss_functions))
    println("bcs_losses: ", map(l_ -> l_(p.u), bcs_inner_loss_functions))
    return false
end

In [ ]:
#Optimizer
opt = OptimizationOptimisers.Adam(0.01)

# We can pass the callback function in the solve. Not doing here as the output would be very long.
res = solve(prob, opt; maxiters = 1000, callback)
phi = discretization.phi

In [ ]:
dx = 0.05
xs, ys = [infimum(d.domain):(dx / 10):supremum(d.domain) for d in domains]
analytic_sol_func(x, y) = (sin(pi * x) * sin(pi * y)) / (2pi^2)

u_predict = reshape([first(phi([x, y], res.u)) for x in xs for y in ys],
    (length(xs), length(ys)))
u_real = reshape([analytic_sol_func(x, y) for x in xs for y in ys],
    (length(xs), length(ys)))
diff_u = abs.(u_predict .- u_real)

p1 = plot(xs, ys, u_real, linetype = :contourf, title = "analytic");
p2 = plot(xs, ys, u_predict, linetype = :contourf, title = "predict");
p3 = plot(xs, ys, diff_u, linetype = :contourf, title = "error");
plot(p1, p2, p3)

In [ ]:
@parameters t, x, y, z
@variables Ex(..), Ey(..), Ez(..), Bx(..), By(..), Bz(..)
E = [Ex, Ey, Ez]
B = [Bx, By, Bz]

Dx = Differential(x)
Dy = Differential(y)
Dz = Differential(z)

sigma = 1e-4
charge = Distributions.MvNormal([0.; 0.; 0.], [1. 0.0 0.0; 0.0 1. 0.0; 0.0 0.00 1.] * sigma)
rho = (x, y, z) -> Distributions.pdf(charge, [x; y; z])

function divergence(f)
    return Dx(f[1](x, y, z)) + Dy(f[2](x, y, z)) + Dz(f[3](x, y, z))
end
function rotor(f)
    ex = Dz(f[2](x, y, z)) - Dy(f[3](x, y, z))
    ey = Dx(f[3](x, y, z)) - Dz(f[1](x, y, z))
    ez = Dy(f[1](x, y, z)) - Dx(f[2](x, y, z))
    return [ex; ey; ez]
end


### Plot of $\rho(\mathbf{r})$ on $Oxy$

In [ ]:
plot(xs, ys, [rho(x, y, 0) for x in xs for y in ys], linetype = :contourf, title = "predict")

In [ ]:
savefig("plot_rho_$sigma.png")

In [ ]:
rotor(E)

In [ ]:
function retrv(f, x, y, z)
    return [f[1](x,y,z); f[2](x,y,z); f[3](x,y,z)]
end

In [ ]:
[(rotor(E)[i] ~ [0.0; 0.0; 0.0][i]) for i in 1:3]

In [ ]:

eqs = cat([
    divergence(E) ~ 4 * pi * rho(x, y, z),
    divergence(B) ~ 0.0
    ], 
    [rotor(E)[i] ~ [0.0; 0.0; 0.0][i] for i in 1:3],
    [rotor(B)[i] ~ [0.0; 0.0; 0.0][i] for i in 1:3]
, dims = 1)


In [ ]:
"""
eqs = [
    divergence(E) ~ 4 * pi * rho(x, y, z),
    divergence(B) ~ 0.0,
    Dz(E[2](x, y, z)) - Dy(E[3](x, y, z)) ~ 0.0,
    Dx(E[3](x, y, z)) - Dz(E[1](x, y, z)) ~ 0.0,
    Dy(E[1](x, y, z)) - Dx(E[2](x, y, z)) ~ 0.0,
    Dz(B[2](x, y, z)) - Dy(B[3](x, y, z)) ~ 0.0,
    Dx(B[3](x, y, z)) - Dz(B[1](x, y, z)) ~ 0.0,
    Dy(B[1](x, y, z)) - Dx(B[2](x, y, z)) ~ 0.0
]
"""

In [ ]:
lower = -2.1
upper = 2.1
bcs = cat(
[E[i](lower, y, z) ~ 0.0 for i in 1:3], [E[i](upper, y, z) ~ 0.0 for i in 1:3],
[B[i](lower, y, z) ~ 0.0 for i in 1:3], [B[i](upper, y, z) ~ 0.0 for i in 1:3],

[E[i](x, lower, z) ~ 0.0 for i in 1:3], [E[i](x, upper, z) ~ 0.0 for i in 1:3],
[B[i](x, lower, z) ~ 0.0 for i in 1:3], [B[i](x, upper, z) ~ 0.0 for i in 1:3],

[E[i](x, y, lower) ~ 0.0 for i in 1:3], [E[i](x, y, upper) ~ 0.0 for i in 1:3],
[B[i](x, y, lower) ~ 0.0 for i in 1:3], [B[i](x, y, upper) ~ 0.0 for i in 1:3],
dims = 1)

In [ ]:
"""
lower = 10
upper = -10
bcs = [Ex(lower, y, z) ~ 0.0, Ex( 1, y, z) ~ 0.0, 
Ey(lower, y, z) ~ 0.0, Ey( 1, y, z) ~ 0.0, 
Ez(lower, y, z) ~ 0.0, Ez( 1, y, z) ~ 0.0, 

Ex(x, lower, z) ~ 0.0, Ex( x, 1, z) ~ 0.0, 
Ey(x, lower, z) ~ 0.0, Ey( x, 1, z) ~ 0.0, 
Ez(x, lower, z) ~ 0.0, Ez( x, 1, z) ~ 0.0, 

Ex(x, y, 0) ~ 0.0, Ex( x, y, 1) ~ 0.0, 
Ey(x, y, 0) ~ 0.0, Ey( x, y, 1) ~ 0.0, 
Ez(x, y, 0) ~ 0.0, Ez( x, y, 1) ~ 0.0, 


Bx(lower, y, z) ~ 0.0, Bx( 1, y, z) ~ 0.0, 
By(lower, y, z) ~ 0.0, By( 1, y, z) ~ 0.0, 
Bz(lower, y, z) ~ 0.0, Bz( 1, y, z) ~ 0.0, 

Bx(x, lower, z) ~ 0.0, Bx( x, 1, z) ~ 0.0, 
By(x, lower, z) ~ 0.0, By( x, 1, z) ~ 0.0, 
Bz(x, lower, z) ~ 0.0, Bz( x, 1, z) ~ 0.0, 

Bx(x, y, 0) ~ 0.0, Bx( x, y, 1) ~ 0.0, 
By(x, y, 0) ~ 0.0, By( x, y, 1) ~ 0.0, 
Bz(x, y, 0) ~ 0.0, Bz( x, y, 1) ~ 0.0]
"""

In [ ]:
const gpud = gpu_device()
const cpud = cpu_device()
# Space and time domains
domains = [x ∈ Interval(lower, upper),
    y ∈ Interval(lower, upper),
    z ∈ Interval(lower, upper)]

# Neural network
input_ = length(domains)
n = 16
chain = [Chain(Dense(input_, n, σ), Dense(n, n, σ), Dense(n, n, σ), Dense(n, 1)) for _ in 1:6]
ps = [Lux.setup(Random.default_rng(), chain[i])[1] |> ComponentArray |> gpud .|> Float64
for i in 1:6]
strategy = QuasiRandomTraining(4096)
discretization = PhysicsInformedNN(chain, strategy; init_params = ps)

In [ ]:
@named pdesystem = PDESystem(eqs, bcs, domains, [x, y, z], [Ex(x, y, z), Ey(x, y, z),Ez(x, y, z), Bx(x, y, z), By(x, y, z), Bz(x, y, z)])

In [ ]:
prob = discretize(pdesystem, discretization)
sym_prob = symbolic_discretize(pdesystem, discretization)

In [ ]:
pde_inner_loss_functions = sym_prob.loss_functions.pde_loss_functions
bcs_inner_loss_functions = sym_prob.loss_functions.bc_loss_functions

callback = function (p, l)
    println("loss: ", l)
    println("pde_losses: ", map(l_ -> l_(p.u), pde_inner_loss_functions))
    println("bcs_losses: ", map(l_ -> l_(p.u), bcs_inner_loss_functions))
    return false
end


In [ ]:

res = solve(prob, OptimizationOptimisers.Adam(1e-2); maxiters = 800, callback, log_frequency = 50)
prob = remake(prob, u0 = res.u)
res = solve(prob, OptimizationOptimisers.Adam(1e-3); maxiters = 800, callback, log_frequency = 50)
prob = remake(prob, u0 = res.u)
res = solve(prob, OptimizationOptimisers.Adam(1e-4); maxiters = 800, callback, log_frequency = 50)

In [ ]:
#prob = remake(prob, u0 = res.u)
#res = solve(prob, LBFGS(linesearch = BackTracking()); maxiters = 1000, callback)

In [ ]:
res.u

In [ ]:
phi = discretization.phi |> cpud

In [ ]:
using Plots

phi = discretization.phi
xs, ys, zs = [infimum(d.domain):0.01:supremum(d.domain) for d in domains]

minimizers_ = [res.u.depvar[sym_prob.depvars[i] ]|> cpud for i in 1:6]
z_selected = 0.4
function analytic_sol_func(t, x, y, z)
    r = sqrt((x) ^ 2 + (y) ^ 2 + (z) ^ 2)
    [(x) / (r^3), (y) / (r^3), (z) / (r^3)]
end
clip = 10
u_real = [[clamp(analytic_sol_func(0, xs, ys, z_selected)[i], -clip, clip) for xs in xs for ys in ys] for i in 1:3]

In [ ]:
u_predict = [[(phi[i]([x, y, z_selected], minimizers_[i]))[1] for x in xs for y in ys] for i in 1:6]

In [ ]:
diff_u = [abs.(u_real[i] .- u_predict[i]) for i in 1:3]

In [ ]:

ps = []
for i in 1:3
    p1 = plot(xs, ys, u_real[i], linetype = :contourf, title = "u$i, analytic")
    p2 = plot(xs, ys, u_predict[i], linetype = :contourf, title = "predict")
    p3 = plot(xs, ys, diff_u[i], linetype = :contourf, title = "error")
    push!(ps, plot(p1, p2, p3))
end

# Решение для $\sigma = \cdot 10^{-1}$

In [ ]:
savefig(ps[1], "figures/plot_Ex_$sigma.png")
ps[1]

In [ ]:
savefig(ps[2], "figures/plot_Ey_$sigma.png")
ps[2]

In [ ]:
savefig(ps[3], "figures/plot_Ez_$sigma.png")
ps[3]

In [ ]:
plt = plot(xs, ys, u_predict[4], linetype = :contourf, title = "predict")
savefig(plt, "figures/plot_Bx_$sigma.png")
plt

# Решение для $\sigma = 2 \cdot 10^{-2}$

## $E_x$

In [ ]:
savefig(ps[1], "figures/plot_Ex_$sigma.png")
ps[1]

## $E_y$

In [ ]:
savefig(ps[2], "figures/plot_Ey_$sigma.png")
ps[2]

## $E_z$

In [ ]:
savefig(ps[3], "figures/plot_Ez_$sigma.png")
ps[3]

# $B_x$

In [ ]:
plt = plot(xs, ys, u_predict[4], linetype = :contourf, title = "predict")
savefig(plt, "figures/plot_Bx_$sigma.png")
plt

# Решение для $\sigma = 5 \cdot 10^{-3}$

In [ ]:
savefig(ps[1], "figures/plot_Ex_$sigma.png")
ps[1]

In [ ]:
savefig(ps[2], "figures/plot_Ey_$sigma.png")
ps[2]

In [ ]:
savefig(ps[3], "figures/plot_Ez_$sigma.png")
ps[3]

In [ ]:
plt = plot(xs, ys, u_predict[4], linetype = :contourf, title = "predict")
savefig(plt, "figures/plot_Bx_$sigma.png")
plt

# Решение для $\sigma = \cdot 10^{-4}$

In [ ]:
savefig(ps[1], "figures/plot_Ex_$sigma.png")
ps[1]

In [ ]:
savefig(ps[2], "figures/plot_Ey_$sigma.png")
ps[2]

In [ ]:
savefig(ps[3], "figures/plot_Ez_$sigma.png")
ps[3]

In [ ]:
plt = plot(xs, ys, u_predict[4], linetype = :contourf, title = "predict")
savefig(plt, "figures/plot_Bx_$sigma.png")
plt

In [ ]:
using JLD2

In [ ]:
res_cpu = (res.u |> cpud)

In [ ]:
@save "res_model_trained.jld2" res_cpu

In [ ]:
@load "res_model_trained.jld2" res_cpu

In [ ]:
res_cpu

In [ ]:
using Plots

phi = discretization.phi
xs, ys, zs = [infimum(d.domain):0.01:supremum(d.domain) for d in domains]

minimizers_ = [res_cpu.depvar[sym_prob.depvars[i] ] for i in 1:6]
z_selected = 0.5
function analytic_sol_func(t, x, y, z)
    r = sqrt((x) ^ 2 + (y) ^ 2 + (z) ^ 2)
    [(x) / (r^3), (y) / (r^3), (z) / (r^3)]
end

u_real = [[clamp(analytic_sol_func(0, xs, ys, z_selected)[i], -2, 2) for xs in xs for ys in ys] for i in 1:3]
u_predict = [[(phi[i]([x, y, z_selected], minimizers_[i]))[1] for x in xs for y in ys] for i in 1:6]
diff_u = [abs.(u_real[i] .- u_predict[i]) for i in 1:3]

In [ ]:

ps = []
for i in 1:3
    p1 = plot(xs, ys, u_real[i], linetype = :contourf, title = "u$i, analytic")
    p2 = plot(xs, ys, u_predict[i], linetype = :contourf, title = "predict")
    p3 = plot(xs, ys, diff_u[i], linetype = :contourf, title = "error")
    push!(ps, plot(p1, p2, p3))
end

In [ ]:
ps[3]

In [ ]:
using NeuralPDE, Lux, ModelingToolkit, Optimization, OptimizationOptimJL
using ModelingToolkit: Interval
using Plots



## Решение волнового уравнения на плоскости

In [ ]:
@parameters x y
@variables u(..)
Dxx = Differential(x)^2
Dyy = Differential(y)^2

# 2D PDE
eq = Dxx(u(x, y)) + Dyy(u(x, y)) ~ -sin(pi * x) * sin(pi * y)

# Boundary conditions
bcs = [u(0, y) ~ 0.0, u(1, y) ~ 0.0,
    u(x, 0) ~ 0.0, u(x, 1) ~ 0.0]
# Space and time domains
domains = [x ∈ Interval(0.0, 1.0),
    y ∈ Interval(0.0, 1.0)]

In [ ]:
# Neural network
dim = 2 # number of dimensions
chain = Chain(Dense(dim, 16, σ), Dense(16, 16, σ), Dense(16, 1))

In [ ]:
discretization = PhysicsInformedNN(
    chain, QuadratureTraining(; batch = 200, abstol = 1e-6, reltol = 1e-6))

In [ ]:
@named pde_system = PDESystem(eq, bcs, domains, [x, y], [u(x, y)])
prob = discretize(pde_system, discretization)

In [ ]:
#Optimizer
opt = OptimizationOptimJL.LBFGS(linesearch = BackTracking())

callback = function (p, l)
    println("Current loss is: $l")
    return false
end

# We can pass the callback function in the solve. Not doing here as the output would be very long.
res = Optimization.solve(prob, opt, maxiters = 1000, callback = callback)
phi = discretization.phi

In [ ]:
dx = 0.05
xs, ys = [infimum(d.domain):(dx / 10):supremum(d.domain) for d in domains]
analytic_sol_func(x, y) = (sin(pi * x) * sin(pi * y)) / (2pi^2)

u_predict = reshape([first(phi([x, y], res.u)) for x in xs for y in ys],
    (length(xs), length(ys)))
u_real = reshape([analytic_sol_func(x, y) for x in xs for y in ys],
    (length(xs), length(ys)))
diff_u = abs.(u_predict .- u_real)

p1 = plot(xs, ys, u_real, linetype = :contourf, title = "analytic");
p2 = plot(xs, ys, u_predict, linetype = :contourf, title = "predict");
p3 = plot(xs, ys, diff_u, linetype = :contourf, title = "error");
plot(p1, p2, p3)

## Решение уравнения для заряда на плоскости

In [ ]:
@parameters t, x, y, z
@variables Ex(..), Ey(..), Ez(..), Bx(..), By(..), Bz(..)
E = [Ex, Ey, Ez]
B = [Bx, By, Bz]

Dx = Differential(x)
Dy = Differential(y)
Dz = Differential(z)

sigma = 1e-4
charge = Distributions.MvNormal([0.; 0.; 0.], [1. 0.0 0.0; 0.0 1. 0.0; 0.0 0.00 1.] * sigma)
rho = (x, y, z) -> Distributions.pdf(charge, [x; y; z])

function divergence(f)
    return Dx(f[1](x, y, z)) + Dy(f[2](x, y, z)) + Dz(f[3](x, y, z))
end
function rotor(f)
    ex = Dz(f[2](x, y, z)) - Dy(f[3](x, y, z))
    ey = Dx(f[3](x, y, z)) - Dz(f[1](x, y, z))
    ez = Dy(f[1](x, y, z)) - Dx(f[2](x, y, z))
    return [ex; ey; ez]
end


### Plot of $\rho(\mathbf{r})$ on $Oxy$

In [ ]:
plot(xs, ys, [rho(x, y, 0) for x in xs for y in ys], linetype = :contourf, title = "predict")

In [ ]:
savefig("plot_rho_$sigma.png")

In [ ]:
rotor(E)

In [ ]:
function retrv(f, x, y, z)
    return [f[1](x,y,z); f[2](x,y,z); f[3](x,y,z)]
end

In [ ]:
[(rotor(E)[i] ~ [0.0; 0.0; 0.0][i]) for i in 1:3]

In [ ]:

eqs = cat([
    divergence(E) ~ 4 * pi * rho(x, y, z),
    divergence(B) ~ 0.0
    ], 
    [rotor(E)[i] ~ [0.0; 0.0; 0.0][i] for i in 1:3],
    [rotor(B)[i] ~ [0.0; 0.0; 0.0][i] for i in 1:3]
, dims = 1)


In [ ]:
"""
eqs = [
    divergence(E) ~ 4 * pi * rho(x, y, z),
    divergence(B) ~ 0.0,
    Dz(E[2](x, y, z)) - Dy(E[3](x, y, z)) ~ 0.0,
    Dx(E[3](x, y, z)) - Dz(E[1](x, y, z)) ~ 0.0,
    Dy(E[1](x, y, z)) - Dx(E[2](x, y, z)) ~ 0.0,
    Dz(B[2](x, y, z)) - Dy(B[3](x, y, z)) ~ 0.0,
    Dx(B[3](x, y, z)) - Dz(B[1](x, y, z)) ~ 0.0,
    Dy(B[1](x, y, z)) - Dx(B[2](x, y, z)) ~ 0.0
]
"""

In [ ]:
lower = -2.1
upper = 2.1
bcs = cat(
[E[i](lower, y, z) ~ 0.0 for i in 1:3], [E[i](upper, y, z) ~ 0.0 for i in 1:3],
[B[i](lower, y, z) ~ 0.0 for i in 1:3], [B[i](upper, y, z) ~ 0.0 for i in 1:3],

[E[i](x, lower, z) ~ 0.0 for i in 1:3], [E[i](x, upper, z) ~ 0.0 for i in 1:3],
[B[i](x, lower, z) ~ 0.0 for i in 1:3], [B[i](x, upper, z) ~ 0.0 for i in 1:3],

[E[i](x, y, lower) ~ 0.0 for i in 1:3], [E[i](x, y, upper) ~ 0.0 for i in 1:3],
[B[i](x, y, lower) ~ 0.0 for i in 1:3], [B[i](x, y, upper) ~ 0.0 for i in 1:3],
dims = 1)

In [ ]:
"""
lower = 10
upper = -10
bcs = [Ex(lower, y, z) ~ 0.0, Ex( 1, y, z) ~ 0.0, 
Ey(lower, y, z) ~ 0.0, Ey( 1, y, z) ~ 0.0, 
Ez(lower, y, z) ~ 0.0, Ez( 1, y, z) ~ 0.0, 

Ex(x, lower, z) ~ 0.0, Ex( x, 1, z) ~ 0.0, 
Ey(x, lower, z) ~ 0.0, Ey( x, 1, z) ~ 0.0, 
Ez(x, lower, z) ~ 0.0, Ez( x, 1, z) ~ 0.0, 

Ex(x, y, 0) ~ 0.0, Ex( x, y, 1) ~ 0.0, 
Ey(x, y, 0) ~ 0.0, Ey( x, y, 1) ~ 0.0, 
Ez(x, y, 0) ~ 0.0, Ez( x, y, 1) ~ 0.0, 


Bx(lower, y, z) ~ 0.0, Bx( 1, y, z) ~ 0.0, 
By(lower, y, z) ~ 0.0, By( 1, y, z) ~ 0.0, 
Bz(lower, y, z) ~ 0.0, Bz( 1, y, z) ~ 0.0, 

Bx(x, lower, z) ~ 0.0, Bx( x, 1, z) ~ 0.0, 
By(x, lower, z) ~ 0.0, By( x, 1, z) ~ 0.0, 
Bz(x, lower, z) ~ 0.0, Bz( x, 1, z) ~ 0.0, 

Bx(x, y, 0) ~ 0.0, Bx( x, y, 1) ~ 0.0, 
By(x, y, 0) ~ 0.0, By( x, y, 1) ~ 0.0, 
Bz(x, y, 0) ~ 0.0, Bz( x, y, 1) ~ 0.0]
"""

In [ ]:
const gpud = gpu_device()
const cpud = cpu_device()
# Space and time domains
domains = [x ∈ Interval(lower, upper),
    y ∈ Interval(lower, upper),
    z ∈ Interval(lower, upper)]

# Neural network
input_ = length(domains)
n = 16
chain = [Chain(Dense(input_, n, σ), Dense(n, n, σ), Dense(n, n, σ), Dense(n, 1)) for _ in 1:6]
ps = [Lux.setup(Random.default_rng(), chain[i])[1] |> ComponentArray |> gpud .|> Float64
for i in 1:6]
strategy = QuasiRandomTraining(4096)
discretization = PhysicsInformedNN(chain, strategy; init_params = ps)

In [ ]:
@named pdesystem = PDESystem(eqs, bcs, domains, [x, y, z], [Ex(x, y, z), Ey(x, y, z),Ez(x, y, z), Bx(x, y, z), By(x, y, z), Bz(x, y, z)])

In [ ]:
prob = discretize(pdesystem, discretization)
sym_prob = symbolic_discretize(pdesystem, discretization)

In [ ]:
pde_inner_loss_functions = sym_prob.loss_functions.pde_loss_functions
bcs_inner_loss_functions = sym_prob.loss_functions.bc_loss_functions

callback = function (p, l)
    println("loss: ", l)
    println("pde_losses: ", map(l_ -> l_(p.u), pde_inner_loss_functions))
    println("bcs_losses: ", map(l_ -> l_(p.u), bcs_inner_loss_functions))
    return false
end


In [ ]:

res = solve(prob, OptimizationOptimisers.Adam(1e-2); maxiters = 800, callback, log_frequency = 50)
prob = remake(prob, u0 = res.u)
res = solve(prob, OptimizationOptimisers.Adam(1e-3); maxiters = 800, callback, log_frequency = 50)
prob = remake(prob, u0 = res.u)
res = solve(prob, OptimizationOptimisers.Adam(1e-4); maxiters = 800, callback, log_frequency = 50)

In [ ]:
#prob = remake(prob, u0 = res.u)
#res = solve(prob, LBFGS(linesearch = BackTracking()); maxiters = 1000, callback)

In [ ]:
res.u

In [ ]:
phi = discretization.phi |> cpud

In [ ]:
using Plots

phi = discretization.phi
xs, ys, zs = [infimum(d.domain):0.01:supremum(d.domain) for d in domains]

minimizers_ = [res.u.depvar[sym_prob.depvars[i] ]|> cpud for i in 1:6]
z_selected = 0.4
function analytic_sol_func(t, x, y, z)
    r = sqrt((x) ^ 2 + (y) ^ 2 + (z) ^ 2)
    [(x) / (r^3), (y) / (r^3), (z) / (r^3)]
end
clip = 10
u_real = [[clamp(analytic_sol_func(0, xs, ys, z_selected)[i], -clip, clip) for xs in xs for ys in ys] for i in 1:3]

In [ ]:
u_predict = [[(phi[i]([x, y, z_selected], minimizers_[i]))[1] for x in xs for y in ys] for i in 1:6]

In [ ]:
diff_u = [abs.(u_real[i] .- u_predict[i]) for i in 1:3]

In [ ]:

ps = []
for i in 1:3
    p1 = plot(xs, ys, u_real[i], linetype = :contourf, title = "u$i, analytic")
    p2 = plot(xs, ys, u_predict[i], linetype = :contourf, title = "predict")
    p3 = plot(xs, ys, diff_u[i], linetype = :contourf, title = "error")
    push!(ps, plot(p1, p2, p3))
end

# Решение для $\sigma = \cdot 10^{-1}$

In [ ]:
savefig(ps[1], "figures/plot_Ex_$sigma.png")
ps[1]

In [ ]:
savefig(ps[2], "figures/plot_Ey_$sigma.png")
ps[2]

In [ ]:
savefig(ps[3], "figures/plot_Ez_$sigma.png")
ps[3]

In [ ]:
plt = plot(xs, ys, u_predict[4], linetype = :contourf, title = "predict")
savefig(plt, "figures/plot_Bx_$sigma.png")
plt

# Решение для $\sigma = 2 \cdot 10^{-2}$

## $E_x$

In [ ]:
savefig(ps[1], "figures/plot_Ex_$sigma.png")
ps[1]

## $E_y$

In [ ]:
savefig(ps[2], "figures/plot_Ey_$sigma.png")
ps[2]

## $E_z$

In [ ]:
savefig(ps[3], "figures/plot_Ez_$sigma.png")
ps[3]

# $B_x$

In [ ]:
plt = plot(xs, ys, u_predict[4], linetype = :contourf, title = "predict")
savefig(plt, "figures/plot_Bx_$sigma.png")
plt

# Решение для $\sigma = 5 \cdot 10^{-3}$

In [ ]:
savefig(ps[1], "figures/plot_Ex_$sigma.png")
ps[1]

In [ ]:
savefig(ps[2], "figures/plot_Ey_$sigma.png")
ps[2]

In [ ]:
savefig(ps[3], "figures/plot_Ez_$sigma.png")
ps[3]

In [ ]:
plt = plot(xs, ys, u_predict[4], linetype = :contourf, title = "predict")
savefig(plt, "figures/plot_Bx_$sigma.png")
plt

# Решение для $\sigma = \cdot 10^{-4}$

In [ ]:
savefig(ps[1], "figures/plot_Ex_$sigma.png")
ps[1]

In [ ]:
savefig(ps[2], "figures/plot_Ey_$sigma.png")
ps[2]

In [ ]:
savefig(ps[3], "figures/plot_Ez_$sigma.png")
ps[3]

In [ ]:
plt = plot(xs, ys, u_predict[4], linetype = :contourf, title = "predict")
savefig(plt, "figures/plot_Bx_$sigma.png")
plt

In [ ]:
using JLD2

In [ ]:
res_cpu = (res.u |> cpud)

In [ ]:
@save "res_model_trained.jld2" res_cpu

In [ ]:
@load "res_model_trained.jld2" res_cpu

In [ ]:
res_cpu

In [ ]:
using Plots

phi = discretization.phi
xs, ys, zs = [infimum(d.domain):0.01:supremum(d.domain) for d in domains]

minimizers_ = [res_cpu.depvar[sym_prob.depvars[i] ] for i in 1:6]
z_selected = 0.5
function analytic_sol_func(t, x, y, z)
    r = sqrt((x) ^ 2 + (y) ^ 2 + (z) ^ 2)
    [(x) / (r^3), (y) / (r^3), (z) / (r^3)]
end

u_real = [[clamp(analytic_sol_func(0, xs, ys, z_selected)[i], -2, 2) for xs in xs for ys in ys] for i in 1:3]
u_predict = [[(phi[i]([x, y, z_selected], minimizers_[i]))[1] for x in xs for y in ys] for i in 1:6]
diff_u = [abs.(u_real[i] .- u_predict[i]) for i in 1:3]

In [ ]:

ps = []
for i in 1:3
    p1 = plot(xs, ys, u_real[i], linetype = :contourf, title = "u$i, analytic")
    p2 = plot(xs, ys, u_predict[i], linetype = :contourf, title = "predict")
    p3 = plot(xs, ys, diff_u[i], linetype = :contourf, title = "error")
    push!(ps, plot(p1, p2, p3))
end

In [ ]:
ps[3]

In [ ]:
# Добавление новой нейросети для моделирования распределения заряда и тока
charge_current_chain = Chain(Dense(3, 16, σ), Dense(16, 16, σ), Dense(16, 4))
charge_current_params = Lux.setup(Random.default_rng(), charge_current_chain)[1] |> ComponentArray |> gpu_device() |> Float64

In [ ]:
# Обновление Loss с учетом новой сети и регуляризации
function custom_loss(p, l)
    # Существующий Loss
    pde_loss = sum(map(l_ -> l_(p.u), pde_inner_loss_functions))
    bc_loss = sum(map(l_ -> l_(p.u), bcs_inner_loss_functions))
    
    # Регуляризация для новой сети
    reg_loss = sum(abs2, charge_current_params)
    
    return pde_loss + bc_loss + 1e-4 * reg_loss
end

In [ ]:
# Обновление пайплайна для обучения новой сети
callback = function (p, l)
    println("loss: ", l)
    println("pde_losses: ", map(l_ -> l_(p.u), pde_inner_loss_functions))
    println("bcs_losses: ", map(l_ -> l_(p.u), bcs_inner_loss_functions))
    return false
end

res = solve(prob, OptimizationOptimisers.Adam(1e-2); maxiters = 800, callback, log_frequency = 50, loss = custom_loss)